##### 0.1. Importing all the needed libraries

In [1]:
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split as tts
from torch.utils.data import Dataset, DataLoader
from torch import nn
from torch import optim
import numpy as np
import pandas as pd
import torch
import optuna
import os

##### 0.2. Setting some global settins

In [2]:
%matplotlib ipympl

##### 0.3. Importing the data file


##### 1.1. Building the Neural Network

In [3]:
def clp_network(trial=False, n_layers=-1, units=[]):
    
    custom_network = not trial and len(units) == n_layers

    if not custom_network: n_layers = trial.suggest_int("n_layers", 2, 5)
    layers = []

    in_features = 6
    
    for i in range(n_layers):
        
        if custom_network: out_features = units[i]
        else: out_features = trial.suggest_int("n_units_l{}".format(i), 4, 10)
        
        layers.append(nn.Linear(in_features, out_features))
        layers.append(nn.LeakyReLU())

        in_features = out_features
        
    layers.append(nn.Linear(in_features, 2))
    layers.append(nn.LeakyReLU())
    
    return nn.Sequential(*layers)

##### 1.2. Making a Dataset class

In [4]:
class clp_dataset(Dataset):

    def __init__(self, df):
        self.labels = [label for label in df['loyalty']]
        self.features = df.drop(columns=['loyalty'], axis=1).values.tolist()

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        return np.array(self.labels[idx])

    def get_batch_features(self, idx):
        return np.array(self.features[idx])

    def __getitem__(self, idx):
        batch_features = self.get_batch_features(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_features, batch_y

##### 1.3. Training and evaluation of the model

In [5]:
def train_and_evaluate(param, model, trial=False):

    # extracting parameters
    if 'n_epochs' in param: n_epochs = param['n_epochs']
    else: n_epochs = 50 # DEFAULT VALUE

    if 'batch_size' in param: batch_size = param['batch_size']
    else: batch_size = 32 # DEFAULT VALUE
    
    # importing data from file
    data_path = os.path.join(os.getcwd(), 'Data\\processed_customer_data.csv')
    data = pd.read_csv(data_path)
    data.drop(columns='ID', inplace=True)
    
    train_data, val_data = tts(data, test_size = 0.2)
    train, val = clp_dataset(train_data), clp_dataset(val_data)

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=batch_size)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    criterion = nn.CrossEntropyLoss()
    optimizer = getattr(optim, param['optimizer'])(model.parameters(), lr= param['learning_rate'])

    if use_cuda:

            model = model.cuda()
            criterion = criterion.cuda()

    for epoch_num in range(n_epochs):

            total_acc_train = 0
            total_loss_train = 0

            for train_input, train_label in train_dataloader:

                train_label = train_label.to(device)
                train_input = train_input.to(device)

                output = model(train_input.float())
                
                batch_loss = criterion(output, train_label.long())
                total_loss_train += batch_loss.item()
                
                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc

                model.zero_grad()
                batch_loss.backward()
                optimizer.step()
            
            total_acc_val = 0
            total_loss_val = 0

            with torch.no_grad():

                for val_input, val_label in val_dataloader:

                    val_label = val_label.to(device)
                    val_input = val_input.to(device)

                    output = model(val_input.float())

                    batch_loss = criterion(output, val_label.long())
                    total_loss_val += batch_loss.item()
                    
                    acc = (output.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc
            
            accuracy = total_acc_val/len(val_data)
            
            # Add prune mechanism
            if trial:
                trial.report(accuracy, epoch_num)

                if trial.should_prune():
                    raise optuna.exceptions.TrialPruned()

    return accuracy, model

##### 1.4. Making the objective function

In [6]:
models = dict()
def objective(trial, n_epochs, batch_size):
     params = {
              'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 1e-1),
              'optimizer': trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"]),
              'n_epochs': n_epochs,
              'batch_size': batch_size
              }
     model = clp_network(trial)
     accuracy, model = train_and_evaluate(params, model, trial)
     if accuracy > models['accuracy']:
          models['accuracy'] = accuracy
          models['model'] = model
     
     return accuracy

In [7]:
bests = []

epoch_step = 30
epoch_count = 5
batch_count = 5
batch_start = 4
epoch_sizes = [i*epoch_step for i in range(1,epoch_count+1)]
batch_sizes = [batch_start * (2**i) for i in range(batch_count)]
study_index = 0
for e in epoch_sizes:
    for b in batch_sizes:
        models = dict()
        models['accuracy'] = 0
        study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(), pruner=optuna.pruners.MedianPruner())
        study.optimize(lambda trial: objective(trial, e, b), n_trials=25
                        , show_progress_bar=True)
        best_params = study.best_trial
        param_dict = best_params.params

        params_dict = dict()

        best_model_index = best_params.number
        params_dict['model'] = models['model']

        params_dict['study_number'] = study_index
        study_index += 1

        params_dict['params'] = param_dict
        params_dict['accuracy'] = best_params.value
        params_dict['n_epoch'] = e
        params_dict['batch_size'] = b

        bests.append(params_dict)

print(f"Study report:\n\t"
    + f"epoch steps = {epoch_step}, epoch counts = {epoch_count}\n\t"
    + f"batch start = {batch_start}, batch counts = {batch_count}", end='\n\n')

for i in range(len(bests)):
    print(f"Study {i}:\tn_epoch = {bests[i]['n_epoch']}, batch_size = {bests[i]['batch_size']}")
    print(f"\tAccuracy = {bests[i]['accuracy']}")

[I 2022-08-30 17:15:57,148] A new study created in memory with name: no-name-bad6570a-7be4-40ee-87bf-590643691cf8
c:\Users\Mohsen\anaconda3\envs\CLP\lib\site-packages\optuna\progress_bar.py:47: ExperimentalWarning: Progress bar is experimental (supported from v1.2.0). The interface can change in the future.
  self._init_valid()


  0%|          | 0/25 [00:00<?, ?it/s]

[I 2022-08-30 17:16:12,862] Trial 0 finished with value: 0.2785714285714286 and parameters: {'learning_rate': 1.4809108109381629e-05, 'optimizer': 'RMSprop', 'n_layers': 2, 'n_units_l0': 8, 'n_units_l1': 8}. Best is trial 0 with value: 0.2785714285714286.
[I 2022-08-30 17:16:29,582] Trial 1 finished with value: 0.7571428571428571 and parameters: {'learning_rate': 3.254690586590479e-05, 'optimizer': 'RMSprop', 'n_layers': 3, 'n_units_l0': 10, 'n_units_l1': 10, 'n_units_l2': 6}. Best is trial 1 with value: 0.7571428571428571.
[I 2022-08-30 17:16:52,485] Trial 2 finished with value: 0.8428571428571429 and parameters: {'learning_rate': 0.0008010846474800237, 'optimizer': 'RMSprop', 'n_layers': 5, 'n_units_l0': 8, 'n_units_l1': 9, 'n_units_l2': 8, 'n_units_l3': 7, 'n_units_l4': 5}. Best is trial 2 with value: 0.8428571428571429.
[I 2022-08-30 17:17:22,148] Trial 3 finished with value: 0.7857142857142857 and parameters: {'learning_rate': 0.004031366440002547, 'optimizer': 'Adam', 'n_layers':

[I 2022-08-30 17:19:35,175] A new study created in memory with name: no-name-476c3221-ccfc-43ab-8b08-8f0546916769


[I 2022-08-30 17:19:35,168] Trial 24 pruned. 


...

[I 2022-08-30 19:01:24,489] A new study created in memory with name: no-name-5cb92921-51ce-4efe-b99f-0417ca3c7363


[I 2022-08-30 19:01:24,485] Trial 24 pruned. 


c:\Users\Mohsen\anaconda3\envs\CLP\lib\site-packages\optuna\progress_bar.py:47: ExperimentalWarning: Progress bar is experimental (supported from v1.2.0). The interface can change in the future.
  self._init_valid()


  0%|          | 0/25 [00:00<?, ?it/s]

[I 2022-08-30 19:01:31,594] Trial 0 finished with value: 0.7571428571428571 and parameters: {'learning_rate': 0.0031961931036177706, 'optimizer': 'SGD', 'n_layers': 5, 'n_units_l0': 9, 'n_units_l1': 10, 'n_units_l2': 6, 'n_units_l3': 8, 'n_units_l4': 10}. Best is trial 0 with value: 0.7571428571428571.
[I 2022-08-30 19:01:40,601] Trial 1 finished with value: 0.7785714285714286 and parameters: {'learning_rate': 0.0005017497384116035, 'optimizer': 'RMSprop', 'n_layers': 5, 'n_units_l0': 9, 'n_units_l1': 10, 'n_units_l2': 4, 'n_units_l3': 9, 'n_units_l4': 6}. Best is trial 1 with value: 0.7785714285714286.
[I 2022-08-30 19:01:46,381] Trial 2 finished with value: 0.6714285714285714 and parameters: {'learning_rate': 0.031405737786068186, 'optimizer': 'SGD', 'n_layers': 3, 'n_units_l0': 7, 'n_units_l1': 10, 'n_units_l2': 10}. Best is trial 1 with value: 0.7785714285714286.
[I 2022-08-30 19:01:53,468] Trial 3 finished with value: 0.2571428571428571 and parameters: {'learning_rate': 0.00241952

In [8]:
max_index = 0
for i in range(1, len(bests)):
    if bests[i]['accuracy'] > bests[max_index]['accuracy']: max_index = i

best_case = bests[max_index]
best_params = best_case['params']
best_model = best_case['model']

torch.save({
    'model_state_dict': best_model.state_dict(),
    'accuracy': best_case['accuracy'],
    'architecture': best_params 
    }, 'Models\\customer_loyalty_prediction.pt')

print(f"Final model:\n\t"
    + f"Accuracy: {best_case['accuracy']}\n\t"
    + f"Model Saved Path: ")

Final model:
	Accuracy: 0.8642857142857143
	Model Saved Path: 
